# Solución al Scrapeo de Mercado Libre
El primer paso es importar las 3 librerias fundamentales para el scrapeo con beautifulsoup.

In [149]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 

En este caso; decidí hacer una prueba con el departamento de celulares de mercadolibre. Este código que sigue se va a contruir de poco en poco para ver que podamos importar los datos que queremos de cada página y ver si somos capaces de encontrar enlaces que nos lleven a las siguientes páginas donde podemos encontrar más productos.

In [150]:
URL1 = 'https://celulares.mercadolibre.com.mx'
URL1 = BeautifulSoup(requests.get(URL1).text, "lxml")

A continuación encontramos los tags de Names (Nombre de producto), Price (Precio actual del producto), Nextpage (enlaces a las siguientes páginas de la sección de celulares) y Sale (Número de productos vendidos.

La parte de discount se utiliza por que no en todos los productos hay un descuento; en ese caso el número de descuentos sería menor al número de precios lo cual no nos ayuda mucho en el la realización de la base de datos. Lo que se hace es invetigar cada producto uno por uno y ver si hay una sección de price-old que indica descuento. Si existe; entonces sustituimos el precio en la lista descuentos pero si no entonces especificamos que no hay descuento.

El siguiente código solo analiza la primera página de mercadolibre.

In [156]:
Name=[row.get_text() for row in URL1.find_all('span',class_='main-title')]
Price=[row.get_text() for row in URL1.find_all('span',class_='price__fraction')]
Nextpage=[row['href'] for row in URL1.find_all('a',class_='andes-pagination__link',href=True)]
Sale=[row.get_text() for row in URL1.find_all('div',class_='item__condition')]
#Solución no elegante, pero funcional
Discount=[]
for item in URL1.find_all('li',class_='results-item'):
    x=item.find_all('span',class_='price-old')
    if x==[]:
        Discount.append('No Discount')
    else:
        for row in x:
            Discount.append(row.get_text())
'''Al obtener los enlaces notamos que los primeros 2 no nos
sirven ya que nos hacen regresar o nos hacen quedarnos en donde estamos.'''
# El último corresponde a la flecha de adelante.
del Nextpage[0]
del Nextpage[0]
del Nextpage[-1]
print(len(Price), len(Discount))

53 53


Una vez hecho eso; hacemos un for en donde se hace todo lo anterior; comenzamos con un request y luego extraemos la información de esa página por lo cual estámos navegando una lista de direcciones web y extrayendo la información de cada una!

In [152]:
for URL in Nextpage:
    Scrap = BeautifulSoup(requests.get(URL).text, "lxml")
    Names=[row.get_text() for row in Scrap.find_all('span',class_='main-title')]
    Prices=[row.get_text() for row in Scrap.find_all('span',class_='price__fraction')]
    Sales=[row.get_text() for row in URL1.find_all('div',class_='item__condition')]
    Discounts=[]
    for item in URL1.find_all('li',class_='results-item'):
        x=item.find_all('span',class_='price-old')
        if x==[]:
            Discounts.append('No Discount')
        else:
            for row in x:
                Discounts.append(row.get_text())
    Name=Name+Names
    Price=Price+Prices
    Discount=Discount+Discounts
    Sale=Sale+Sales

Después podemos hacer la limpieza de los datos; eliminamos texto no deseado, podemos pasar de strings a números para poder hacer operaciones sobre ellos más facilmente y finalmente terminamos por crear un dataframe con los parámetros deseados.

In [161]:
#Limpieza, símbolos y transformación.
Price=[p.replace(',','') for p in Price]
Price=[int(p) for p in Price]
Dollar=list(map(lambda x: x/20,Price))

In [162]:
df=pd.DataFrame({'Name':Name,'Price':Price,'Discount':Discount,'Sales':Sale,'Dollar':Dollar})
df.to_csv('Mercado.csv')
df

,Name,Price,Discount,Sales,Dollar
0,Xiaomi Mi A2 Lite 64+4gb Global 12+5+5mpx Nuevo,3870,No Discount,1836 vendidos,193.50
1,Celular Moto E4 16gb Ram 2gb Quad Core Androi...,1749,"$ 1,999",757 vendidos,87.45
2,Celular Samsung Galaxy M20 32gb Ram 3gb Super...,3588,"$ 3,987",3662 vendidos,179.40
3,Celular Samsung Galaxy M10 16gb Ram 2gb Nuevo...,2567,"$ 2,853",964 vendidos,128.35
4,Huawei Y9 2019 64gb Dual Sim 3gb Ram Camara D...,4849,No Discount,353 vendidos,242.45
5,Celular Apple iPhone 6 16gb 4g Lte Demo,3399,"$ 4,999",866 vendidos,169.95
6,Celular Samsung Galaxy M30 64gb 4gb R 3 Camar...,5199,"$ 5,777",2354 vendidos,259.95
7,Huawei P30 128gb + 6gb Ram + Regalo Watch Gt,15949,No Discount,138 vendidos,797.45
8,Celular Motorola Moto E5 Play 16gb Ram 2gb 5.2,1998,"$ 2,220",325 vendidos,99.90
9,Celular Lg V20 64gb 16mp Android 7 Liberado D...,2925,"$ 3,999",920 vendidos,146.25


# Ejercicio palacio de hierro
Comenzamos por hacer un request por la página del departamento de bolsos de palacio de hierro.

In [137]:
URL2 = 'https://www.elpalaciodehierro.com/lujo/mujer/accesorios/bolsos.html'
URL2 = BeautifulSoup(requests.get(URL2).text, "lxml")

Ahora imprimimos el código html en la consola para ver que hayamos obtenido una página con información y no algo vacío. De hecho palacio de hierro incluye algo muy curioso en su página de internet.

In [148]:
URL2.index

<bound method Tag.index of <!DOCTYPE html>
<html class="no-js" id="top" lang="en">
<head>
<!--

          # ,@;  @.  ':
               @@     @
          ,    +@#  `@    ;:
               +  '#;    #@
             '@@@@       @,
          `@@:   `      @@
         +@@   `.      :@
        '@@  .#  ;@    @#
        @@  :     :@  #@
       :@:         @  @;
       #@  `       @ @@       .;;    ;+
       '@         @@.@` #   @#. `#@@#  `
        @        :@ @@ #  #+      :;@.                                                                                .
        @;      .@  @ ;  @       :@  @                                                           `#@@'.      `@.     ;
         @@.  .@+   @@  @       `@   `@                                                         @     ;@@@@@@@:    :#
          ,++;         @        @@    @`                                                       +           ;@;    ;@
                      @        @@     @#                                               

Ahora vamos a extraer la información relevante del documento html por medio de una comprensión de lista; inclusive se hace el filtrado de la variable precio ya que cuenta con un caracter adicional que no es de nuestro interés y puede terminar complicando el análisis de los datos.

La variable next page palacio cuenta con los enlaces de las siguientes páginas del catálogo de bolsas Palacio.

In [138]:
MarcaPalacio=[row.get_text().strip() for row in URL2.find_all('p',class_='ls-grid-title')]
NamePalacio=[row['title'] for row in URL2.find_all('span',title=True)]
PricePalacio=[row.get_text().strip() for row in URL2.find_all('span',class_='price')]
PricePalaciofilt=[word.replace(u'\xa0', '') for word in PricePalacio]
NextpagePalacio=['https://www.elpalaciodehierro.com' + link['href'] for link in URL2.find_all('a',class_='nume',href=True)]
NextpagePalacio=NextpagePalacio[0:3]
print(NextpagePalacio)


['https://www.elpalaciodehierro.com/lujo/mujer/accesorios/bolsos.html?p=2', 'https://www.elpalaciodehierro.com/lujo/mujer/accesorios/bolsos.html?p=3', 'https://www.elpalaciodehierro.com/lujo/mujer/accesorios/bolsos.html?p=4']


Con esto podemos hacer un ciclo for para scrappear esas páginas y juntar los resultados en una gran lista individual.

In [94]:
for link in NextpagePalacio:
    URL = BeautifulSoup(requests.get(link).text, "lxml")
    
    MarcaPalacios=[row.get_text().strip() for row in URL2.find_all('p',class_='ls-grid-title')]
    NamePalacios=[row['title'] for row in URL.find_all('span',title=True)]
    PricePalacios=[row.get_text().strip() for row in URL.find_all('span',class_='price')]
    PricePalaciosfilt=[word.replace(u'\xa0', '') for word in PricePalacios]
    
    NamePalacio=NamePalacio+NamePalacios
    PricePalaciofilt=PricePalaciofilt+PricePalaciosfilt
    MarcaPalacio=MarcaPalacio+MarcaPalacios

['Cosmetiquera', 'Bolso de mano Gancini', 'Bolso mini con lazo Vara', 'Bolso crossbody en piel', 'Bolso satchel en piel', 'Bolso pequeño', 'Bolso satchel en piel', 'Bolso lazo Vara con solapa frontal', 'Ferragamo Studio bag', 'Bolso de Mano Gancini', 'Bolso de Mano Gancini', 'Bolso crossbody animal skin', 'Bolso Sofia', 'Tote Ferragamo Studio bag', 'Bolso compacto Gancini', 'Bolso Gancini con solapa frontal', 'Bolso mini lazo Vara', 'Bolso shopper con grafiti de piel', 'Bolso crossbody de piel con animal print', 'Bolso crossbody de piel', 'Bolso shoulder de piel', 'Bolso tote de piel con animal print', 'Bolso tote de piel', 'Bolso backpack de piel', 'Bolso satchel de piel', 'Bolso tote en piel', 'Bolso Whip Piel', 'Bolso Cangurera Piel', 'Bolso crossbody de piel granulada', 'Bolso Whip Piel', 'Bolso Whip Piel', 'Bolso satchel de piel', 'Bolso Gem Piel Capitonado', 'Bolso Pouch algodón Flores', 'Bolso Bucket Piel', 'Bolso crossbody de piel granulada', 'Bolso Bucket Piel', 'Bolso Whip Pi

Para terminar; solo metemos las listas en un diccionario y terminamos exportandola en un csv que podemos encontrar en la misma carpeta que este documento.

In [135]:
df=pd.DataFrame({'Marca':MarcaPalacio,'Nombre':NamePalacio,'Precio':PricePalacioc})
df.to_csv('Palacio.csv')

# Scraping de Soriana
La última página por scrapear es Soriana; nuevamente escogemos un departamento de nuestro interés que puede ser el de pantallas de Soriana por decir. Pueden usar cualquier otro que gusten ya que la estructura es la misma. Comenzamos por usar la misma instrucción

In [109]:
URL2 = 'https://www.soriana.com/soriana/es/c/Electronica/Pantallas/Pantallas/c/111'
URL2 = BeautifulSoup(requests.get(URL2).text, "lxml")

['/soriana/es/c/Electr%C3%B3nica/Pantallas/Pantallas/c/111?q=%3Arelevance&page=1', '/soriana/es/c/Electr%C3%B3nica/Pantallas/Pantallas/c/111?q=%3Arelevance&page=2']


In [133]:
NomSoriana=[Nom.get_text().strip() for Nom in URL2.find_all('a',class_="name")]
PrecioSoriana=[precio.get_text().strip() for precio in URL2.find_all('span',class_='price')]
DiscountSoriana=[]
for item in URL2.find_all('div',class_='priceContainer'):
    x=item.find_all('span',class_='sale-original-price')
    if x==[]:
        DiscountSoriana.append('Igual')
    else:
        for row in x:
            DiscountSoriana.append(row.get_text())
for links in URL2.find_all('ul',class_='pagination'):
    enlaces=['https://www.soriana.com'+link['href'] for link in links.find_all('a',class_='',href=True)]
print(enlaces)

['https://www.soriana.com/soriana/es/c/Electr%C3%B3nica/Pantallas/Pantallas/c/111?q=%3Arelevance&page=1', 'https://www.soriana.com/soriana/es/c/Electr%C3%B3nica/Pantallas/Pantallas/c/111?q=%3Arelevance&page=2']


In [134]:
for enlace in enlaces:
    Scrap = BeautifulSoup(requests.get(enlace).text, "lxml")
    NomSorianas=[Nom.get_text().strip() for Nom in Scrap.find_all('a',class_="name")]
    PrecioSorianas=[precio.get_text().strip() for precio in Scrap.find_all('span',class_='price')]
    DiscountsSoriana=[]
    for item in Scrap.find_all('div',class_='priceContainer'):
        x=item.find_all('span',class_='sale-original-price')
        if x==[]:
            DiscountsSoriana.append('Igual')
        else:
            for row in x:
                DiscountsSoriana.append(row.get_text())
    DiscountSoriana=DiscountSoriana+DiscountsSoriana
    NomSoriana=NomSoriana+NomSorianas
    PrecioSoriana=PrecioSoriana+PrecioSorianas
print(DiscountSoriana)

['$15,999.00', '$11,999.00', '$29,999.00', '$19,999.00', '$26,999.00', '$3,899.00', '$49,999.00', '$34,999.00', '$6,999.00', '$24,999.00', '$5,999.00', '$5,499.00', '$88,999.00', '$16,499.00', '$11,889.00', '$10,999.00', '$13,799.00', '$18,499.00', '$18,399.00', '$15,499.00', '$19,999.00', '$13,999.00', '$18,999.00', 'Igual', '$15,999.00', '$15,999.00', '$11,799.00', '$13,499.00', '$13,999.00', '$9,999.00', '$8,199.00', '$23,999.00', '$15,699.00', '$16,999.00', '$25,999.00', '$35,999.00', '$11,679.00', '$7,899.00', '$6,790.00', '$17,999.00', '$6,999.00', '$5,299.00', '$19,999.00', '$14,999.00', '$46,999.00', '$4,499.00', '$3,149.00', 'Igual', 'Igual', '$4,999.00', '$7,999.00', '$27,799.00', '$25,999.00', '$6,999.00']


In [ ]:
DatosSor=pd.DataFrame({'Product':NomSoriana, 'Precio Original':DiscountSoriana, 'Precio Descontado':PrecioSoriana})